<a href="https://colab.research.google.com/github/arsengir/colab/blob/main/machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка данных

https://www.kaggle.com/datasets/abdelrahmankhalil/100000-uk-used-car-data-set

https://drive.google.com/file/d/17xk2fS4QevT2NKziTD2puH9w9B8qXpCZ/view?usp=sharing

In [ ]:
! gdown --id 17xk2fS4QevT2NKziTD2puH9w9B8qXpCZ

In [ ]:
! unzip /content/uk_used_cars.zip -d /content/uk_used_cars

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/uk_used_cars/bmw.csv')

In [ ]:
df.head()

# Обучающая и тестовая выборка (train and test)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(df, train_size = 0.6, random_state=42)

# Валидационная выборка 

In [ ]:
len(train)/len(df)

In [ ]:
val, test = train_test_split(test, train_size = 0.5, random_state=42)

In [ ]:
len(train)/len(df)

In [ ]:
len(val)/len(df)

In [ ]:
len(test)/len(df)

# Список фичей

In [ ]:
train.columns

In [ ]:
train

In [ ]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']
cat_features = ['model', 'transmission', 'fuelType']       
y = ['price']

# Принцип преобразования категориальных фичей

In [ ]:
train['transmission']

In [ ]:
pd.get_dummies(train['transmission'])

https://catboost.ai/en/docs/concepts/algorithm-main-stages_cat-to-numberic

# Первый запуск Кэтбуста

In [ ]:
! pip install catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
X = ['year', 'transmission', 'engineSize']
cat_features = ['transmission']       
y = ['price']

https://catboost.ai/en/docs/concepts/python-reference_utils_eval_metric

In [ ]:
model = CatBoostRegressor(cat_features=cat_features, 
                          eval_metric='MAPE', 
                          random_seed=42,
                          verbose=100)

In [ ]:
model.fit(train[X],train[y], eval_set=(val[X],val[y]))

In [ ]:
model.predict(test[X])

In [ ]:
test['price_pred'] = model.predict(test[X])

In [ ]:
test

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [ ]:
def error(y_true, y_pred):
    print(mean_absolute_error(y_true, y_pred))
    print(mean_absolute_percentage_error(y_true, y_pred))

In [ ]:
error(test['price'],test['price_pred'])

# Обучение на всех фичах

In [ ]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']
cat_features = ['model', 'transmission', 'fuelType']       
y = ['price']

In [ ]:
parametrs = {'cat_features': cat_features,
             'eval_metric': 'MAPE',
             'random_seed': 42,
             'verbose': 100}

In [ ]:
model = CatBoostRegressor(**parametrs)

In [ ]:
model.fit(train[X], train[y], eval_set=(val[X],val[y]))

In [ ]:
test['price_pred_all'] = model.predict(test[X])

In [ ]:
error(test['price'],test['price_pred_all'])

# Количество итераций и learning rate

In [ ]:
X = ['year', 'transmission', 'engineSize']
cat_features = ['transmission']       
y = ['price']

In [ ]:
model = CatBoostRegressor(cat_features=cat_features, 
                          learning_rate=0.03,
                          eval_metric='MAPE', 
                          random_seed=42,
                          verbose=100)

In [ ]:
model.fit(train[X],train[y], eval_set=(val[X],val[y]))

In [ ]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']
cat_features = ['model', 'transmission', 'fuelType']       
y = ['price']

In [ ]:
parametrs = {'cat_features': cat_features,
             'eval_metric': 'MAPE',
             'learning_rate': 0.08,
             'random_seed': 42,
             'verbose': 100}

In [ ]:
model = CatBoostRegressor(**parametrs)

In [ ]:
model.fit(train[X], train[y], eval_set=(val[X],val[y]))

In [ ]:
X = ['year', 'transmission', 'engineSize']
cat_features = ['transmission']       
y = ['price']

In [ ]:
model = CatBoostRegressor(cat_features=cat_features, 
                          early_stopping_rounds=200,
                          eval_metric='MAPE', 
                          random_seed=42,
                          verbose=100)

In [ ]:
model.fit(train[X],train[y], eval_set=(val[X],val[y]))

# Обучаемся на всех данных

In [ ]:
train_full = pd.concat([train, val])

In [ ]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']
cat_features = ['model', 'transmission', 'fuelType']       
y = ['price']

In [ ]:
parametrs = {'cat_features': cat_features,
             'eval_metric': 'MAPE',
             'learning_rate': 0.08,
             'random_seed': 42,
             'verbose': 100}

In [ ]:
model = CatBoostRegressor(**parametrs)

In [ ]:
model.fit(train[X], train[y], eval_set=(val[X],val[y]))

In [ ]:
model.best_iteration_

In [ ]:
parametrs = {'iterations': model.best_iteration_+1,
             'cat_features': cat_features,
             'eval_metric': 'MAPE',
             'learning_rate': 0.08,
             'random_seed': 42,
             'verbose': 100}

In [ ]:
model = CatBoostRegressor(**parametrs)

In [ ]:
model.fit(train_full[X], train_full[y])

In [ ]:
test['price_pred_all_features_and_data'] = model.predict(test[X])

In [ ]:
error(test['price'], test['price_pred_all_features_and_data'])

# Метрики регрессии

https://catboost.ai/en/docs/concepts/loss-functions-regression

# Меняем функцию оптимизации

In [ ]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']
cat_features = ['model', 'transmission', 'fuelType']       
y = ['price']

In [ ]:
parametrs = {'cat_features': cat_features,
             'loss_function': 'MAE',
             'eval_metric': 'MAPE',
             'learning_rate': 0.1,
             'random_seed': 42,
             'verbose': 100}

In [ ]:
model = CatBoostRegressor(**parametrs)

In [ ]:
model.fit(train[X], train[y], eval_set=(val[X],val[y]))